In [ ]:
#integration Code

In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model

# Load YOLO configuration and weights
yolo_weights = "./yolov3.weights"
yolo_cfg = "./yolov3.cfg"
coco_names = "./coco.names"

# Load the YOLO model
net = cv2.dnn.readNet(yolo_weights, yolo_cfg)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load classes
with open(coco_names, "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load the trained model for age prediction
loaded_model = load_model("age_prediction_sigmoid256_modelkeras.keras")

# Output directory for saving faces
output_folder = "./result"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Image dimensions for preprocessing
image_width, image_height = 256, 256

def preprocess_image(image):
    img = cv2.resize(image, (image_width, image_height))  # Resize to 256x256
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    return img

# Start webcam capture
cap = cv2.VideoCapture(0)  # 0 for default camera
print("Press 'c' to capture an image and process it, or 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    cv2.imshow("Press 'c' to Capture", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):  # Capture frame on 'c' key
        height, width = frame.shape[:2]
        
        # Prepare the image for YOLO
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outputs = net.forward(output_layers)
        
        faces = []
        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                
                if class_id == 0 and confidence > 0.4:  # Detect faces
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = max(0, int(center_x - w / 2))
                    y = max(0, int(center_y - h / 2))
                    faces.append((x, y, w, h))

        # If multiple faces, choose the leftmost
        if faces:
            leftmost_face = min(faces, key=lambda b: b[0])  # Select face with smallest x-coordinate
            x, y, w, h = leftmost_face
            face = frame[y:y + h, x:x + w]

            if face.size > 0:
                # Preprocess the face for the model
                preprocessed_face = preprocess_image(face)
                
                # Predict age
                prediction = loaded_model.predict(preprocessed_face)[0][0]
                predicted_label = int(round(prediction))  # 0 or 1
                label_text = "underage" if predicted_label == 0 else "notunderage"
                
                # Save face with label
                save_name = f"captured_{label_text}_{len(os.listdir(output_folder)) + 1}.jpg"
                save_path = os.path.join(output_folder, save_name)
                resized_face = cv2.resize(face, (image_width, image_height))
                cv2.imwrite(save_path, resized_face)
                
                print(f"Saved labeled face: {save_path}")
        else:
            print("No faces detected. Try again.")

    elif key == ord('q'):  # Quit on 'q' key
        print("Exiting...")
        break

cap.release()
cv2.destroyAllWindows()


Press 'c' to capture an image and process it, or 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Saved labeled face: ./result\captured_notunderage_9.jpg
Exiting...


In [ ]:
#Instructions:
#Run the Script: Execute the script in a Python environment with OpenCV and TensorFlow installed.
#Capture Images: Press c to capture and process a frame, or q to quit.
#Face Processing:
#Detects all faces in the frame.
#Selects the leftmost face.
#Predicts the age category (underage or notunderage).
#Saves the processed face in the ./result folder.
#This integrates real-time capture with the rest of the pipeline effectively.